# We are going to look at making the GloVE based version of Node2Vec

This is the GloVE as node2vec is to word2vec.

Papers:

 - [GloVE](http://www.aclweb.org/anthology/D14-1162)
 - [node2vec](https://cs.stanford.edu/people/jure/pubs/node2vec-kdd16.pdf)

In [1]:
using Pkg
pkg"activate .."

In [2]:
using SparseArrays

# Now we want to avoid taking random walks

Node2Vec takes random walks, to get sequences of word(nodes), which it feeds to word2vec.

Instread we want to get a matrix that corresponds to the co-occurrencee matrix in GloVE.
Which has elements that are weighted counts of how often words(nodes) occur in the same window.



$A$ is the adjancency matrix.
Nodes directly connected to neighbours

We need to get a co-occurrence matrix $X$,
which has entries given by the count of how often each node occurred in the context of another node.
Where context is determined by window size.

Graph theory tells us for nodes, $i$ and $j$
 - $A_{ij}$ is there a connection between node $i$ and node $j$
 - $(A^2)_{ij}$ number of walks of length 2 from node $i$ to node $j$
 - $(A^n)_{ij}$ number of walks of length n from node $i$ to node $j$
 
 
GLoVE uses decreasing weighting such that words $d$ apart contribute $\frac{1}{d}$ to the coocurance count  (GloVE paper section 4.2).


Thus for window size $N$,

$$
X = \sum_{n=1}^{n=N}   \frac{A^n}{n}
$$


This (I believe) extends intutitively and without change to:
 - using a **weight** matrix instread of an **adjancency** matrix.
 - Or to using a directed graph, via **asymetrical** adjancency matrix
 - Or both.
Normalization may be required.


 

In [3]:
function cooccurance_matrix(W, window_size)
    X = Float32.(W)
    Wp = W
    
    for n in 2 : window_size
        Wp *= W
        X .+= Wp./n
    end
    X
end

cooccurance_matrix (generic function with 1 method)

The downside of this method is that it is fairly expensive.
Matrix multiplication of a matrix with average $k$ nonzero elements
is (slightly better in theory, but roughly) $O(k^3)$.  
Thus for window size $N$,
that the calculation of the cooccurance matrix is
$O(nk^3)$  
Not also that sparsity doesn't help as much as we would like,
and each successive matrix multiplication is less sparse.

## Node2Vec biased walks:
 
 Quoting:  [node2vec paper](https://cs.stanford.edu/people/jure/pubs/node2vec-kdd16.pdf)
 section 3.2.2 Search bias
 
The simplest way to bias our random walks would be to sample the next node based on the static edge weights $w_{vx}$ i.e, $\pi_{vx} = w_{vx}$. (In case of unweighted graphs $w_{vx}=1$.) However, this does not allow us to account for the network structure and guide our search procedure to explore different types of network neighborhoods. 
Additionally, unlike BFS and DFS which are extreme sampling paradigms suited for structural equivalence and homophily respectively, our random walks should accommodate for the fact that these notions of equivalence are not competing or exclusive, and real-world networks commonly exhibit a mixture of both. 

We define a 2$^{\textrm{nd}}$ order random walk with two parameters $p$ and $q$ which guide the walk: Consider a random walk that just traversed edge $(t,v)$ and now resides at node $v$ . The walk now needs to decide on the next step so it evaluates the transition probabilities $\pi_{vx}$ on edges $(v,x)$ leading from $v$. We set the unnormalized transition probability to $\pi_{vx} = \alpha_{pq}(t,x)\cdot w_{vx}$, where 

$$
	\alpha_{pq}(t,x) = 
	\begin{cases}
	\frac{1}{p}  & \text{if } d_{tx} = 0\\
	1 & \text{if } d_{tx} = 1\\
	\frac{1}{q} & \text{if } d_{tx} = 2
	\end{cases}
$$

and $d_{tx}$ denotes the shortest path distance between nodes $t$ and $x$. Note that $d_{tx}$ must be one of $\{0,1,2\}$, and hence, the two parameters are necessary and sufficient to guide the walk.


### Weighting functions

In [4]:
function π_vx(W, t,v,x; p=1, q=1)
    w_vx = @inbounds W[v,x]
    w_vx == 0 && return 0
    π_vx = α(W, t, x; p=p, q=q) * w_vx
end

function α(W, t, x; p=1, q=1)
    if t==x #d_tx=0
        1/p
    elseif @inbounds W[t,x] > 0 #d_tx=1
        1
    else #d_tx=2 as go t->v,v->x
        1/q
    end
end

α (generic function with 1 method)

## Generating the collocation matrix

This is an overload of the `cooccurance_matrix`
to take the extra arguments for `p` and `q`

In [5]:
function cooccurance_matrix(graph_weights, window_size, p=1, q=1)
    W = sparse(graph_weights)
    X = copy(W)
    
    πW1=copy(W) # First step
    for d in 2:window_size
        πW2=spzeros(size(W)...) # whipe it
        for (t, v, val) in zip(findnz(πW1)...)
            xs = SparseArrays.nonzeroinds(@inbounds W[v,:])
            for x in xs
                @inbounds πW2[v, x] += π_vx(W, t,v,x; p=p, q=q)
            end
        end
        πW1 *= πW2
        X .+= πW1/=d # GloVE proximity weighting
    end
    X
end

cooccurance_matrix (generic function with 3 methods)

# Now lets solve GLoVE

Fine matrixes C and V such that
$$
loss = f(X)(CV - \log X)^2
$$
is minimized


we use 

 - Optim as a global optimizer

In [6]:
using Optim

Define the problem.  
We will be storing $C$ and $V$ in the same matrix as that is how Optim likes ot handle memory. We call that combo `CV`.

We also need to define the weighting function $f$ (we use the GLoVE defn here, I don't think it is optimal though.)

And the Loss, which we express as function of X that returns a function of `CV`

In [9]:
function split_CV(CV)
    C = @view(CV[:, 1:end÷2])'
    V = @view(CV[:, end÷2+1:end])
    C,V
end

function f(x)
    xmax = 100
    α = 2/3
    
    x < xmax ? (x/xmax)^α : 1.0
end

loss(X)=function (CV)
    C,V = split_CV(CV)
    sum(f.(X) * (C*V .- log.(X)).^2)
end

loss (generic function with 1 method)

### Now to build the optimiser,  
this will take in the Collocation matrix `X` and the number of dimensions to use.  
This  can be tweaked a fair bit;

kwparams:

 - `time_limit` max time to run for in seconds

In [10]:
function nodeGLoVE(X, num_dims=16; time_limit = 5*60)
    num_dims = 16
    CV = randn(num_dims, 2size(X,1)) # Twice the size as we need to split it off
    res = optimize(loss(X), CV, LBFGS(), 
        Optim.Options(show_trace=true,
            #iterations = 200
            time_limit = time_limit
            ))
    C,V = split_CV(res.minimizer)
end

nodeGLoVE (generic function with 2 methods)